In [2]:
import requests

In [3]:
API_KEY = ' '
HEADERS = {'Authorization': f'Bearer {API_KEY}'}
WEATHER_API_KEY = ' '

## Taking Details of Village The Soul Of India

In [5]:
business_id = 'village-the-soul-of-india-hicksville'
url = f"https://api.yelp.com/v3/businesses/{business_id}"

response = requests.get(url, headers = HEADERS)
data = response.json()

print(data)


{'id': '_sPZ8kWu97HGvVZ-mQIvWg', 'alias': 'village-the-soul-of-india-hicksville', 'name': 'Village the soul of india', 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/bhomw1W-6OQ_hEXgNNfo3g/o.jpg', 'is_claimed': True, 'is_closed': False, 'url': 'https://www.yelp.com/biz/village-the-soul-of-india-hicksville?adjust_creative=qCUrb1JWQH46lVRgJg_8-A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=qCUrb1JWQH46lVRgJg_8-A', 'phone': '+15168590057', 'display_phone': '(516) 859-0057', 'review_count': 46, 'categories': [{'alias': 'vegetarian', 'title': 'Vegetarian'}, {'alias': 'indpak', 'title': 'Indian'}], 'rating': 4.8, 'location': {'address1': '11 West Marie St', 'address2': None, 'address3': '', 'city': 'Hicksville', 'zip_code': '11801', 'country': 'US', 'state': 'NY', 'display_address': ['11 West Marie St', 'Hicksville, NY 11801'], 'cross_streets': ''}, 'coordinates': {'latitude': 40.76656, 'longitude': -73.52353}, 'photos': ['https://s3-media1.fl.yelpcdn.com/bpho

In [6]:
name = data.get('name')
address = ', '.join(data['location']['display_address'])
hours = data.get('hours', [{}])[0].get('open', [])
schedule = [ f"{day['start']} - {day['end']}" for day in hours]
menu_url = data['attributes']['menu_url']
print(f"Name: {name}")
print(f"Address: {address}")
print(f"Hours: {schedule}")

Name: Village the soul of india
Address: 11 West Marie St, Hicksville, NY 11801
Hours: ['1100 - 1500', '1700 - 2200', '1100 - 1500', '1700 - 2200', '1100 - 1500', '1700 - 2200', '1100 - 1500', '1700 - 2200', '1130 - 2200', '1130 - 2200']


In [30]:
from datetime import datetime

def convert_time_format(time_str):
    if time_str:  # Check if time_str is not empty
        time_obj = datetime.strptime(time_str, '%H%M')
        return time_obj.strftime('%I:%M %p')
    else:
        return "Invalid Time"

# Convert the time ranges
formatted_times = []
for time_range in schedule:
    start, end = time_range.split(' - ')
    start_time = convert_time_format(start)
    end_time = convert_time_format(end)
    formatted_times.append(f'{start_time} - {end_time}')
print(formatted_times)

['11:00 AM - 03:00 PM', '05:00 PM - 10:00 PM', '11:00 AM - 03:00 PM', '05:00 PM - 10:00 PM', '11:00 AM - 03:00 PM', '05:00 PM - 10:00 PM', '11:00 AM - 03:00 PM', '05:00 PM - 10:00 PM', '11:30 AM - 10:00 PM', '11:30 AM - 10:00 PM']


In [31]:
print(menu_url)

https://www.villagesoulofindia.com/app/order/menu


In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import random
import pandas as pd
import numpy as np

In [33]:
import re

chrome_options = Options()
chrome_options.add_argument("--lang=en")
prefs = {
    "intl.accept_languages": "en, en_US",
    "gl": "US",
    "hl": "en",
}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
driver.get(menu_url)
menus = driver.find_elements(By.CSS_SELECTOR, "div.card-body h5 b")
village_menu = []
for menu in menus:
    village_menu.append(menu.text)

prices = driver.find_elements(By.CSS_SELECTOR, "div.text-muted")

village_prices  = []
for price in prices:
    village_prices.append(price.text)
    
joined_prices = ' '.join(village_prices)

prices = re.findall(r'\$\s?\d+\.\d{2}', joined_prices)

# Clean the prices (optional, remove any leading whitespace after the $ symbol)
prices = [price.replace(" ", "") for price in prices]
prices = [float(price.replace('$', '')) for price in prices]

village_menu_price = list(zip(village_menu, prices))
village_menu_df = pd.DataFrame(village_menu_price, columns=['Food', 'Price'])
village_menu_df.head(30)

,Food,Price
0,Rasam,4.95
1,Sambar,4.95
2,Mulligatawny Soup,4.95
3,Garden Soup,4.95
4,Kachumbar Salad,4.95
5,Garden Salad,4.95
6,Veg Manchow soup,4.95
7,Tomato Soup,4.95
8,Katchumber Salad,4.95
9,Idly,5.95


## Finding Nearby Restaurants

In [35]:
categories = data['categories']

village_categories = []
for category in categories:
    village_categories.append(category['alias'])

village_categories = ','.join(village_categories)

latitude = data['coordinates']['latitude']
longitude = data['coordinates']['longitude']
print(village_categories, latitude, longitude)

vegetarian,indpak 40.76656 -73.52353


In [36]:
url = "https://api.yelp.com/v3/businesses/search"
params = {
    'term': 'restaurant',                # Search for restaurants
    'categories': village_categories,    # Similar categories
    'latitude': latitude,                # Latitude of the current restaurant
    'longitude': longitude,              # Longitude of the current restaurant
    'radius': 2000,                      # Radius in meters (2 km)
    'sort_by': 'rating',                 # Sort by rating
    'limit': 5                           # Limit to top 5 results
}

response = requests.get(url, headers=HEADERS, params=params)

data = response.json()
# Extract details for top 5 restaurants
top_restaurants = []
for business in data['businesses']:
    top_restaurants.append({
        'name': business['name'],
        'rating': business['rating'],
        'address': ', '.join(business['location']['display_address']),
        'menu_url': business['attributes']['menu_url'],
        'distance': round(business['distance'] / 1000, 2)  # Convert meters to km
    })

# Print the results
for idx, restaurant in enumerate(top_restaurants, start=1):
    print(f"{idx}. {restaurant['name']} - {restaurant['rating']} stars")
    print(f"   Address: {restaurant['address']}")
    print(f"   Distance: {restaurant['distance']} km")
    print(f"   Menu: {restaurant['menu_url']}\n")



1. Village the soul of india - 4.8 stars
   Address: 11 West Marie St, Hicksville, NY 11801
   Distance: 0.0 km
   Menu: https://www.villagesoulofindia.com/app/order/menu

2. Kunga Kitchen - 4.6 stars
   Address: 390 Woodbury Rd, Hicksville, NY 11801
   Distance: 2.44 km
   Menu: None

3. Taste of Mumbai - 4.6 stars
   Address: 153 Levittown Pkwy, Hicksville, NY 11801
   Distance: 2.1 km
   Menu: https://www.tasteofmumbainy.com/menu

4. Kathis & Kababs - 4.4 stars
   Address: 55 Broadway, Ste A, Hicksville, NY 11801
   Distance: 0.21 km
   Menu: https://www.kathiskababs.com/Menu.aspx

5. Taste Of Chennai - 4.4 stars
   Address: 39 W John St, Hicksville, NY 11801
   Distance: 0.45 km
   Menu: https://www.tasteofchennainy.com/menus



In [37]:
# Kunga Kitchen
kunga_url = 'https://www.yelp.com/menu/kunga-kitchen-hicksville'
driver.get(kunga_url)
menus = driver.find_elements(By.CSS_SELECTOR, 'div.arrange_unit.arrange_unit--fill.menu-item-details h4')

prices = driver.find_elements(By.CSS_SELECTOR, 'li.menu-item-price-amount')

kunga_menu = []
for menu in menus:
    kunga_menu.append(menu.text)

kunga_price = []
for price in prices:
    kunga_price.append(price.text.strip())

kunga_price = [float(p.replace('$','')) for p in kunga_price]

kunga_menu_price = list(zip(kunga_menu, kunga_price))

kunga_df = pd.DataFrame(kunga_menu_price, columns = ['Food', 'Price'])
kunga_df.head(10)

,Food,Price
0,CHILLI FISH DRY,15.59
1,CHILLI SHRIMP DRY,15.59
2,VEG SAMOSA,5.19
3,VEG SPRING ROLL,7.27
4,CHICKEN SPRING ROLL,7.27
5,CHICKEN TIKKA,15.59
6,CHICKEN SEEKH KABAB,13.51
7,CAULIFLOWER MANCHURIAN DRY,9.35
8,MASALA FRENCH FRIES,7.27
9,CRISPY HONEY LOTUS,10.39


In [38]:
tom_url = 'https://www.yelp.com/menu/taste-of-mumbai-hicksville-3'
driver.get(tom_url)
menus = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.arrange div h4 a')))

tom_menus = []

for menu in menus:
    tom_menus.append(menu.text)

prices = driver.find_elements(By.CSS_SELECTOR, 'li.menu-item-price-amount')
tom_prices = []
for price in prices:
    tom_prices.append(price.text)

tom_prices = [float(p.replace('$', '')) for p in tom_prices]

tom_menu_prices = list(zip(tom_menus, tom_prices))
tom_df = pd.DataFrame(tom_menu_prices, columns = ['Food', 'Price'])

tom_df.head(10)

,Food,Price
0,Saag Paneer,16.99
1,Malai Kofta,17.99
2,Manchow Soup,17.99
3,Papdi Chaat,17.99
4,Vada Pav,17.99
5,Vegetable Samosa,16.99
6,Chicken Samosa,17.99
7,Spring Roll,17.99
8,Lasuni Gobi,18.99
9,Chicken Tikka,3.99


In [39]:
#kathis&kababs
kk_url = 'https://www.yelp.com/menu/kathis-and-kababs-hicksville'
driver.get(kk_url)
menus = driver.find_elements(By.CSS_SELECTOR, 'div.arrange div h4')
kk_menus = []
for menu in menus:
    kk_menus.append(menu.text.lstrip("1234567890. "))

kk_prices = []
prices = driver.find_elements(By.CSS_SELECTOR, 'li.menu-item-price-amount')
for price in prices:
    kk_prices.append(price.text)

kk_prices = [float(p.replace('$', '')) for p in kk_prices]
kk_menu_prices = list(zip(kk_menus, kk_prices))
kk_df = pd.DataFrame(kk_menu_prices, columns = ['Food', 'Price'])
kk_df.head(10)


,Food,Price
0,Achari Aaloo Kathi Roll,11.95
1,Paneer Tikka Kathi Roll,11.95
2,Mushroom Kathi Roll,11.95
3,Malai Paneer Kathi Roll,11.95
4,Egg Omelette Kathi Roll,11.95
5,Chicken Tikka Kathi Roll,11.95
6,Chicken Malai Kathi Roll,11.95
7,Chicken Achari Kathi Roll,11.95
8,Chicken Seekh Kathi Roll,11.95
9,Lamb Seekh Kathi Roll,14.95


In [40]:
#tasteOfChennai

toc_url = 'https://www.yelp.com/menu/taste-of-chennai-hicksville'
driver.get(toc_url)
menus = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.arrange div h4')))
toc_menus = []

for menu in menus:
    toc_menus.append(menu.text)

toc_prices = []
prices = driver.find_elements(By.CSS_SELECTOR, 'li.menu-item-price-amount')
for price in prices:
    toc_prices.append(price.text)

toc_prices = [float(p.replace('$', '')) for p in toc_prices]
toc_menu_prices = list(zip(toc_menus, toc_prices))

toc_df = pd.DataFrame(toc_menu_prices, columns = ['Food', 'Price'])
toc_df.head(10)

,Food,Price
0,Sambar,7.49
1,Rasam,7.49
2,Chicken Soup,8.99
3,Punjabi Samosa (2 pcs),6.99
4,Onion Pakora,8.99
5,Cut Mirchi (6 pcs),7.49
6,Lasuni Gobi,11.49
7,Chili Paneer,12.49
8,Medhu Vada (2 pcs),7.99
9,Idli (2 pcs),7.99


## Finding The Lowest Price

In [42]:
# Converting menu dfs into dictionaries
village_menu = village_menu_df.set_index('Food')['Price'].to_dict()
kunga_menu = kunga_df.set_index('Food')['Price'].to_dict()
tom_menu = tom_df.set_index('Food')['Price'].to_dict()
kk_menu = kk_df.set_index('Food')['Price'].to_dict()
toc_menu = toc_df.set_index('Food')['Price'].to_dict()

In [43]:
from fuzzywuzzy import fuzz, process

menus = [village_menu, kunga_menu, tom_menu, kk_menu, toc_menu]

# Initialize a dictionary to store the lowest price for each menu item
lowest_prices = {}

# Function to find the best match for an item from all menus
def get_best_match(item_name, all_menus):
    best_match = None
    highest_score = 0
    for menu in all_menus:
        for menu_item in menu:
            score = fuzz.ratio(item_name.lower(), menu_item.lower())  # Calculate the similarity score
            if score > highest_score:
                highest_score = score
                best_match = menu_item
    return best_match

# Iterate through each menu and find the lowest price for each matched item
for menu in menus:
    for item, price in menu.items():
        best_match = get_best_match(item, menus)
        
        if best_match not in lowest_prices:
            lowest_prices[best_match] = price
        else:
            lowest_prices[best_match] = min(lowest_prices[best_match], price)

# Print the lowest prices for matched items
lowest_price = {}
for item, price in lowest_prices.items():
    #print(f"{item}: ${price:.2f}")
    lowest_price[item] = price

lowest_price_df = pd.DataFrame(list(lowest_price.items()), columns=['Food', 'Price'])
lowest_price_df.head(10)


,Food,Price
0,Rasam,4.95
1,Sambar,4.95
2,Mulligatawny Soup,4.95
3,Garden Soup,4.95
4,Kachumbar Salad,4.95
5,Garden Salad,4.95
6,Veg Manchow soup,4.95
7,Tomato Soup,4.95
8,Katchumber Salad,4.95
9,Idly,5.95


## Getting Busy Times

In [45]:
map_url = 'https://www.google.com/maps/place/Village+-+The+Soul+of+India/@40.7665603,-73.523538,17z/data=!4m14!1m7!3m6!1s0x89c281752d83843d:0x1f2a365d2207b71c!2sVillage+-+The+Soul+of+India!8m2!3d40.7665603!4d-73.523538!16s%2Fg%2F11thj448_5!3m5!1s0x89c281752d83843d:0x1f2a365d2207b71c!8m2!3d40.7665603!4d-73.523538!16s%2Fg%2F11thj448_5?entry=ttu&g_ep=EgoyMDI0MTIxMS4wIKXMDSoASAFQAw%3D%3D?hl=en&gl=US'
driver.delete_all_cookies() 
driver = webdriver.Chrome()
driver.get(map_url)

busy_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[role="img"][aria-label]')))

# Extract the aria-label values
busy_times = [element.get_attribute('aria-label') for element in busy_elements]

# Define a threshold for filtering (e.g., discard below 20%)
threshold = 20

# Filter out times when the restaurant is not busy enough
#filtered_busy_times = [
#    time for time in busy_times if int(time.split('%')[0]) >= threshold
#]
times = []
# Print the filtered busy times
for time in busy_times:
    times.append(time)

percentage_pattern = r'(\d+)%'  # Matches the number before a percentage
time_pattern = r'(\d{1,2}\u202f?[APap][Mm])'  # Matches the time (e.g., 6 AM, 6 PM)

# Extract percentages and times
percentages = re.findall(percentage_pattern, " ".join(times))
times = re.findall(time_pattern, " ".join(times))
percentages = [float(p) for p in percentages]
times = [time.replace("\u202f", "") for time in times]
# Print the extracted values

busy_time = []
for i,per in enumerate(percentages):
    if per > 30:
        busy_time.append(times[i])

busy_time = sorted(set(busy_time))
print(busy_time)



['12PM', '1PM', '2PM', '5PM', '6PM', '7PM', '8PM', '9PM']


## Current Temperature And Rain

In [47]:

weather_url = f'https://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={WEATHER_API_KEY}'
response = requests.get(weather_url)
data = response.json()

temp_kelvin = data['main']['temp']
temp_fahrenheit = (temp_kelvin - 273.15) * 9/5 + 32
temp_fahrenheit = round(temp_fahrenheit, 2)
rain = data.get('rain', {}).get('1h', 0)

print(f"Temperature in Fahrenheit : {temp_fahrenheit} F")
print(f"Rain in last one hour : {rain} mm " )

Temperature in Fahrenheit : 39.83 F
Rain in last one hour : 2.35 mm 


## Price Adjustment of Village Restaurant Based On Busy Time And Bad Weather

Let us increase the price of the restaurant's menu by 10% and 15% on busy time and bad weather respectively. Before that, let's set up some required entities.

In [50]:
from datetime import datetime

#Determining whether the current time is in busy time or not
# Get the current time
now = datetime.now()

# Convert the current time to 12-hour format with AM/PM
current_time = now.strftime("%-I%p") 

if current_time in busy_time:
    is_busy = True
else:
    is_busy = False

print(is_busy)
print(current_time)

True
5PM


In [51]:
#Determining whether there is bad weather or not setting some thresholds
temp_threshold = 45
rain_threshold = 2.5

if temp_fahrenheit < temp_threshold:
    is_badWeather = True
elif rain >= rain_threshold:
    is_badWeather = True
else:
    is_badWeather = False

print(is_badWeather)

True


In [52]:
#Multipliers

model_data = []
busy_time_multiplier = 1.10  # 10% price increase during busy times
weather_multiplier = 1.15  # 15% price increase for bad weather

def adjust_price(village_menu, lowest_price_df, is_busy, is_badWeather):
    adjusted_prices = {}
    
    for item, village_price in village_menu.items():
        # Compare the item with the lowest price menu
        lowest_competitor_price = lowest_price_df[lowest_price_df['Food'] == item]['Price']
        
        # If the item is not found in the competitor list, continue
        if lowest_competitor_price.empty:
             lowest_competitor_price = village_price
        else:
             lowest_competitor_price = lowest_competitor_price.iloc[0]  # Get the price value


        # Adjust price based on conditions
        if is_badWeather or is_busy:
            # Apply the price increase (e.g., 10% increase in busy times, 15% for bad weather)
            adjusted_price = village_price * busy_time_multiplier if is_busy else village_price * weather_multiplier
            # Ensure the price is higher than the lowest competitive price
            adjusted_price = max(adjusted_price, lowest_competitor_price * 1.05)  # 5% more than lowest competitor
        else:
            # If no conditions are met, use the lowest competitor price
            adjusted_price = lowest_competitor_price
        
        adjusted_prices[item] = adjusted_price

        

        model_data.append({
            'Food': item,
            'Village_Price': village_price,
            'Competitor_Price': lowest_competitor_price,
            'Temperature_F': temp_fahrenheit,
            'Is_Bad_Weather': is_badWeather,
            'Is_Busy_Time': is_busy,
            'Target_Price': adjusted_price
        })
    return adjusted_prices

adjusted_prices = adjust_price(village_menu, lowest_price_df, is_busy, is_badWeather)

# Display the adjusted prices
#for item, price in adjusted_prices.items():
 #  print(f"{item}: ${price:.2f}")

adjusted_prices_df = pd.DataFrame(list(adjusted_prices.items()), columns=['Food', 'Adjusted_Price'])
adjusted_prices_df.head(5)

,Food,Adjusted_Price
0,Rasam,5.445
1,Sambar,5.445
2,Mulligatawny Soup,5.445
3,Garden Soup,5.445
4,Kachumbar Salad,5.445


## ML MODEL

In [54]:
model_df = pd.DataFrame(model_data)
model_df.head()

,Food,Village_Price,Competitor_Price,Temperature_F,Is_Bad_Weather,Is_Busy_Time,Target_Price
0,Rasam,4.95,4.95,39.83,True,True,5.445
1,Sambar,4.95,4.95,39.83,True,True,5.445
2,Mulligatawny Soup,4.95,4.95,39.83,True,True,5.445
3,Garden Soup,4.95,4.95,39.83,True,True,5.445
4,Kachumbar Salad,4.95,4.95,39.83,True,True,5.445


In [55]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Example: Prepare the dataset
X = model_df[['Village_Price', 'Competitor_Price', 'Temperature_F', 'Is_Bad_Weather', 'Is_Busy_Time']]
y = model_df['Target_Price']

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict adjusted prices
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

# Predict for current conditions
current_conditions = X.copy()  # Assuming conditions are already in X
model_df['Predicted_Price'] = model.predict(current_conditions)

model_df.head()


Mean Squared Error: 1.18


,Food,Village_Price,Competitor_Price,Temperature_F,Is_Bad_Weather,Is_Busy_Time,Target_Price,Predicted_Price
0,Rasam,4.95,4.95,39.83,True,True,5.445,5.445
1,Sambar,4.95,4.95,39.83,True,True,5.445,5.445
2,Mulligatawny Soup,4.95,4.95,39.83,True,True,5.445,5.445
3,Garden Soup,4.95,4.95,39.83,True,True,5.445,5.445
4,Kachumbar Salad,4.95,4.95,39.83,True,True,5.445,5.445


## With Randomized Search

In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from scipy.stats import randint

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model 
model = RandomForestRegressor(random_state=42)

# Define the hyperparameters to tune using RandomizedSearchCV
param_dist = {
    'n_estimators': randint(900, 1300),  
    'max_features': ['sqrt', 'log2', None],  
    'max_depth': randint(10, 100),  
    'min_samples_split': randint(2, 20),  
    'min_samples_leaf': randint(1, 20),  
    'bootstrap': [True, False]  
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, 
                                   n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1, scoring='r2')

# Fit the model with RandomizedSearchCV
random_search.fit(X_train, y_train)

# Get the best parameters found by RandomizedSearchCV
best_params = random_search.best_params_
print(f"Best Parameters: {best_params}")

# Use the best model to predict and evaluate
best_model = random_search.best_estimator_

# Perform Cross-Validation with the best model
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='r2')  # 5-fold cross-validation

# Calculate the training accuracy (R²) on the training data
train_r2 = r2_score(y_train, best_model.predict(X_train))

# Calculate the testing accuracy (R²) on the testing data
test_r2 = r2_score(y_test, best_model.predict(X_test))

# Print the results
print(f"Training Accuracy (R²): {train_r2:.2f}")
print(f"Testing Accuracy (R²): {test_r2:.2f}")
print(f"Cross-Validation Accuracy (R²): {cv_scores.mean():.2f} ± {cv_scores.std():.2f}")


Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Parameters: {'bootstrap': False, 'max_depth': 41, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 11, 'n_estimators': 1285}
Training Accuracy (R²): 1.00
Testing Accuracy (R²): 0.92
Cross-Validation Accuracy (R²): 0.99 ± 0.00


## Without Hyperparameter Tuning

In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model (RandomForestRegressor in this case)
model = RandomForestRegressor(random_state=42)

# Perform Cross-Validation
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')  # 5-fold cross-validation

# Calculate the training accuracy (R²) on the training data
model.fit(X_train, y_train)
train_r2 = r2_score(y_train, model.predict(X_train))

# Calculate the testing accuracy (R²) on the testing data
test_r2 = r2_score(y_test, model.predict(X_test))

# Print the results
print(f"Training Accuracy (R²): {train_r2:.2f}")
print(f"Testing Accuracy (R²): {test_r2:.2f}")
print(f"Cross-Validation Accuracy (R²): {cv_scores.mean():.2f} ± {cv_scores.std():.2f}")


Training Accuracy (R²): 1.00
Testing Accuracy (R²): 0.93
Cross-Validation Accuracy (R²): 0.99 ± 0.01


Decided to use the model without hyperarameter tuning

In [61]:
# Predict for current conditions
current_conditions = X.copy()  # Assuming conditions are already in X
model_df['Predicted_Price'] = model.predict(current_conditions)

model_df.head()


,Food,Village_Price,Competitor_Price,Temperature_F,Is_Bad_Weather,Is_Busy_Time,Target_Price,Predicted_Price
0,Rasam,4.95,4.95,39.83,True,True,5.445,5.445
1,Sambar,4.95,4.95,39.83,True,True,5.445,5.445
2,Mulligatawny Soup,4.95,4.95,39.83,True,True,5.445,5.445
3,Garden Soup,4.95,4.95,39.83,True,True,5.445,5.445
4,Kachumbar Salad,4.95,4.95,39.83,True,True,5.445,5.445
